In [148]:
import json
import math

import numpy as np
import pvl
import spiceypy as spice

In [149]:
# Utility Func
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item

In [150]:
# Furnish the IK Kernel
# spice.furnsh("../../tests/data/msgr_mdis_v160.ti")
# ikid = 236820

# spice.furnsh('../../tests/data/msgr_v231.tf')  # Frames Kernel mapping between frames
# spice.furnsh('../../tests/data/pck00010_msgr_v23.tpc')  # PC Kernel with planetary attitude and body information
# spice.furnsh('../../tests/data/msgr_dyn_v600.tf')

# spice.furnsh('../../tests/data/msgr_de405_de423s.bsp')
# spice.furnsh('../../tests/data/msgr_040803_150430_150430_od431sc_2.bsp')

# spice.furnsh('../../tests/data/msgr_mdis_sc050727_100302_sub_v1.bc')
# spice.furnsh('../../tests/data/msgr_mdis_gm040819_150430v1.bc')  # MDIS Instrument Pointing (236890)
# spice.furnsh('../../tests/data/msgr_1304_v02.bc')  #Bus and Spacecraft Panels position

# spice.furnsh('../../tests/data/naif0011.tls')
# spice.furnsh('../../tests/data/messenger_2548.tsc')

ikid = 236800
msgr = "/usgs/cpkgs/isis3/data/messenger/"
spice.furnsh("/usgs/cpkgs/isis3/data/base/kernels/lsk/naif0012.tls")
spice.furnsh(msgr+"kernels/pck/pck00010_msgr_v23.tpc")
spice.furnsh(msgr+"kernels/tspk/de423s.bsp")
spice.furnsh(msgr+"kernels/ck/msgr20150409.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150410.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150411.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150412.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150413.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150414.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150415.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150416.bc")
spice.furnsh(msgr+"kernels/ck/1072683119_1965_mdis_atthist.bc")
spice.furnsh(msgr+"kernels/ck/1072716050_291010_mdis_pivot_pvtres.bc")
spice.furnsh(msgr+"kernels/fk/msgr_v231.tf")
spice.furnsh(msgr+"kernels/ik/msgr_mdis_v160.ti")
spice.furnsh(msgr+"kernels/sclk/messenger_2548.tsc")
spice.furnsh(msgr+"kernels/spk/msgr_20040803_20150430_od431sc_2.bsp")
spice.furnsh(msgr+"kernels/iak/mdisAddendum009.ti")

In [151]:
# Create the ISD object
isd = {}

# Load information from the IK kernel
isd['focal_length'] = spice.gdpool('INS-{}_FOCAL_LENGTH'.format(ikid), 0, 1)
isd['focal_length_epsilon'] = spice.gdpool('INS-{}_FL_UNCERTAINTY'.format(ikid), 0, 1)
isd['nlines'] = spice.gipool('INS-{}_PIXEL_LINES'.format(ikid), 0, 1)
isd['nsamples'] = spice.gipool('INS-{}_PIXEL_SAMPLES'.format(ikid), 0, 1)
isd['original_half_lines'] = isd['nlines'] / 2.0
isd['original_half_samples'] = isd['nsamples'] / 2.0
isd['pixel_pitch'] = spice.gdpool('INS-{}_PIXEL_PITCH'.format(ikid), 0, 1)
isd['ccd_center'] = spice.gdpool('INS-{}_CCD_CENTER'.format(ikid), 0, 1)
isd['ifov'] = spice.gdpool('INS-{}_IFOV'.format(ikid), 0, 1)
isd['boresight'] = spice.gdpool('INS-{}_BORESIGHT'.format(ikid), 0, 3)
isd['transx'] = spice.gdpool('INS-{}_TRANSX'.format(ikid), 0, 3)
isd['transy'] = spice.gdpool('INS-{}_TRANSY'.format(ikid), 0, 3)
isd['itrans_sample'] = spice.gdpool('INS-{}_ITRANSS'.format(ikid), 0, 3)
isd['itrans_line'] = spice.gdpool('INS-{}_ITRANSL'.format(ikid), 0, 3)
isd['odt_x'] = spice.gdpool('INS-{}_OD_T_X'.format(ikid), 0, 10)
isd['odt_y'] = spice.gdpool('INS-{}_OD_T_Y'.format(ikid), 0, 10)
isd['starting_detector_sample'] = spice.gdpool('INS-{}_FPUBIN_START_SAMPLE'.format(ikid), 0, 1)
isd['starting_detector_line'] = spice.gdpool('INS-{}_FPUBIN_START_LINE'.format(ikid), 0, 1)


What is the minimum information needed to convert to and from the MDIS-NAC frame?

236: The code for the messenger spacecraft
4: Indicates that the frame is a TK Frame
236820: The code for the MDIS-NAC Frame


In [152]:
# Type 4 is TK - the mdis-nac is constant w.r.t. another frame
print(spice.frinfo(-236820))
print(spice.frinfo(-236802))

# Verify that this is messenger
print(spice.bods2c('MESSENGER'))

(-236, 4, -236820)
(-236, 4, -236802)
-236


In [153]:
# Load the ISIS Cube header
#header = pvl.load('../../tests/data/EN1007907102M.cub')
header = pvl.load("../../tests/data/CW1071364100B_IU_5.cub")

isd['instrument_id'] = find_in_dict(header, 'InstrumentId')
isd['spacecraft_name'] = find_in_dict(header, 'SpacecraftName')
isd['target_name'] = find_in_dict(header, 'TargetName')

# Get the radii from SPICE
rad = spice.bodvrd(isd['target_name'], 'RADII', 3)
radii = rad[1]
isd['semi_major_axis'] = rad[1][0]
isd['semi_minor_axis'] = rad[1][1]


Now we need to compute time.

In [167]:
# Here convert the sclock
sclock = find_in_dict(header, 'SpacecraftClockCount')
exposure_duration = find_in_dict(header, 'ExposureDuration')
exposure_duration = exposure_duration.value * 0.001  # Scale to seconds

# Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
spacecraft_id = spice.bods2c('MESSENGER')
print(spacecraft_id)
et = spice.scs2e(spacecraft_id, sclock)
print(et)
et += (exposure_duration / 2.0)

isd['ephemeris_time'] = 482312178.161
print(et)

-236
482312178.15084046
482312178.16084045


In [169]:
# This needs to be sensor origin - like the comment below says...
# This is almost there - position w.r.t. the spacecraft, but not the camera yet.
loc, ltc = spice.spkpos('MESSENGER', isd['ephemeris_time'], 'IAU_MERCURY', 'LT+S', isd['target_name'])
loc *= 1000
print(loc)
isd['x_sensor_origin'] = loc[0]
isd['y_sensor_origin'] = loc[1]
isd['z_sensor_origin'] = loc[2]

[   28889.98086492   633882.71602422  2683307.06317541]


In [156]:
camera2bodyfixed = spice.pxform('MSGR_MDIS_WAC','IAU_MERCURY', isd['ephemeris_time'])
opk = spice.m2eul(camera2bodyfixed, 3, 2, 1)

isd['omega'] = opk[2]
isd['phi'] = opk[1]
isd['kappa'] = opk[0]

In [157]:
camera2bodyfixed

array([[-0.95210748, -0.00279567, -0.30575075],
       [ 0.06553629,  0.97485318, -0.21299361],
       [ 0.29865755, -0.22283058, -0.92798179]])

In [158]:
print(isd['omega'], isd['phi'], isd['kappa'])

2.905930549960423 0.3032856983573016 -3.072868188822314


In [159]:
class NumpyAwareJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray) and obj.ndim == 1:
            lobj = obj.tolist()
            if len(lobj) == 1:
                return lobj[0]
            else:
                return lobj
        return json.JSONEncoder.default(self, obj)

with open('isd.isd', 'w') as f:
    f.write(json.dumps(isd, f, cls=NumpyAwareJSONEncoder))

In [160]:
!cat isd.isd

{"transy": [0.0, 1.32495711261385e-07, 0.013997772676294], "focal_length": 78.2367, "omega": 2.905930549960423, "focal_length_epsilon": 0.15, "transx": [0.0, 0.01399999999972, 0.0], "starting_detector_sample": 9.0, "original_half_lines": 512.0, "z_sensor_origin": 2683307.06317541, "semi_major_axis": 2439.4, "semi_minor_axis": 2439.4, "ephemeris_time": 482312178.161, "x_sensor_origin": 28889.980864917787, "original_half_samples": 512.0, "nlines": 1024, "odt_x": [0.0, 1.0, 0.0, -7.720894252056575e-05, 3.599871902138938e-06, 0.0, 5.509035727272325e-06, 0.0, 5.509035727272406e-06, 0.0], "nsamples": 1024, "kappa": -3.072868188822314, "instrument_id": "MDIS-WAC", "pixel_pitch": 0.014, "odt_y": [0.0, 0.0, 1.000000000026148, 0.0, -7.720894252092194e-05, 3.599871782473616e-06, 0.0, 5.509035621941527e-06, 0.0, 5.5090308738198125e-06], "ifov": 179.6, "starting_detector_line": 1.0, "y_sensor_origin": 633882.716024222, "spacecraft_name": "Messenger", "ccd_center": 512.5, "target_name": "Mercury", "

In [161]:
# This is hard coded in Anne's script
isisFocalPlan2SocetPlate = np.eye(3)
isisFocalPlan2SocetPlate[1,1] = -1.0
isisFocalPlan2SocetPlate[2,2] = -1.0

# Grab the body fixed coordinates from SPICE

# The mercury Naif ID code is 199
nid = 199


In [162]:
# OPK
isd['x_sensor_origin'] = 
isd['y_sensor_origin'] = 
isd['z_sensor_origin'] = 
isd['omega'] = 
isd['phi'] =
isd['kappa'] = 

SyntaxError: invalid syntax (<ipython-input-162-69367ee46c92>, line 2)

In [ ]:
# ISD Search Information - totally fabricated.
isd['min_elevation'] = -1.0
isd['max_elevation'] = 1.0

In [ ]:
isd

In [ ]:
# Get the sun's position relative to a Mercury-fixed frame.
target = "SUN"
et = isd['ephemeris_time']
reference_frame = "IAU_MERCURY"
light_time_correction = "LT+S"
observer = "MERCURY"

sun_state, lt = spice.spkezr(target, et, reference_frame, light_time_correction, observer)
isd['x_sun_position'] = sun_state[0] * 1000
isd['y_sun_position'] = sun_state[1] * 1000
isd['z_sun_position'] = sun_state[2] * 1000
print("SUN POSITION (m): {} {} {}".format(sun_state[0]*1000, sun_state[1]*1000, sun_state[2]*1000))

# Get velocity of mdis nac (right now it is getting velocity of spacecraft, not sensor)
target = "MESSENGER"
et = isd['ephemeris_time']
reference_frame = "IAU_MERCURY"
light_time_correction = "LT+S"
observer = "MERCURY"
messenger_state, lt = spice.spkezr(target, et, reference_frame, light_time_correction, observer)
print("MESSENGER VELOCITY (m/s): {} {} {}".format(messenger_state[3]*1000, messenger_state[4]*1000, messenger_state[5]*1000))